In [1]:
source('solarslope_function.R')

In [2]:
#site<-"NZSINR"
#lat<--41.90 
#lon<-171.44
#height<-0.5
#incline<-21
#aspect<-170
#albedo<-0.1

In [31]:
site<-"NZSIBT"
lat<--43.58
lon<-172.78
height<-0.5
incline<-14
aspect<-290
albedo<-0.1

In [32]:
heights <- seq(0.5, 3.5, 0.1)

In [33]:
basefolder <- paste('../data/', site, '/', sep='')

In [34]:
readcfsr <- function(basefolder, var){    
    varFolder<-paste(basefolder, "cfsr_vars/", var, "/", sep="")
    varFiles<-list.files(path=varFolder, pattern=NULL, full.names=FALSE, recursive=FALSE)

    for(a in 1:length(varFiles)){
        vardata<-read.table(paste(varFolder, varFiles[a], sep=""), header=TRUE)
        assign(varFiles[a], vardata)
    }

    allvardata<-do.call("rbind", lapply(varFiles, as.name))

    return(allvardata)
}

In [35]:
air<-readcfsr(basefolder, 'tmp2m')
dlwrf<-readcfsr(basefolder, 'dlwsfc')
dswrf<-readcfsr(basefolder, 'dswsfc')
prate<-readcfsr(basefolder, 'prate')
pres<-readcfsr(basefolder, 'pressfc')
rhum<-readcfsr(basefolder, 'rh2m')
uwnd<-readcfsr(basefolder, 'uwnd10m')
vwnd<-readcfsr(basefolder, 'vwnd10m')

In [36]:
weather<-merge(air, dlwrf, all=TRUE)
weather<-merge(weather, prate, all=TRUE)
weather<-merge(weather, pres, all=TRUE)
weather<-merge(weather, rhum, all=TRUE)
weather<-merge(weather, uwnd, all=TRUE)
weather<-merge(weather, vwnd, all=TRUE)

In [37]:
weather<-weather[order(weather$Unix),]
colnames(weather)<-c("Unix", "jday", "air", "dlwrf", "prate", "pres", "rhum", "uwnd", "vwnd")

In [38]:
weather$wnd<-sqrt(weather$uwnd^2+weather$vwnd^2)
weather$prate<-weather$prate*1800/25.4
weather$pres<-weather$pres/100
weather$air<-weather$air-273
weather<-round(weather, digits=2)

In [39]:
colnames(dswrf) <- c('Unix', 'jday', 'dswrf')
solar_slope<-solarslope(lat, lon, incline, aspect, albedo, dswrf[,c('Unix', 'dswrf')]) 
weather<-merge(weather, solar_slope, all=TRUE)

In [40]:
DateConvert<-weather$Unix
class(DateConvert)<-"POSIXct"
DateExtract<-as.POSIXlt(DateConvert)
weather$Date<-as.character(DateExtract, format="%Y-%m-%d")
weather<-weather[order(weather$Unix),]
duration<-length(unique(weather$Date))
Ephemeris<-read.table(pipe(paste("~/Code/BaseCode/ephemeris/eph", " --lat ", lat, " --lon ", lon, "  --date ", weather$Date[1], " --duration ", duration, " --time 0")))
Ephemeris<-Ephemeris[,c(1,2,5)]
colnames(Ephemeris)<-c("Date", "T", "Elev")
Ephemeris$T<-round(as.numeric(Ephemeris$T), digits=1)
Ephemeris$Hour<-floor(as.numeric(Ephemeris$T))
Ephemeris$Minute<-(as.numeric(Ephemeris$T)-floor(as.numeric(Ephemeris$T)))*60
Ephemeris$Minute<-round(Ephemeris$Minute, digits=0)
Ephemeris$DateTime<-paste(Ephemeris$Date, " ", Ephemeris$Hour, ":", Ephemeris$Minute, sep="")
Ephemeris<-subset(Ephemeris, Ephemeris$Minute==0 | Ephemeris$Minute==30)
store<-strptime(Ephemeris$DateTime, "%Y-%m-%d %H:%M")
store2<-as.POSIXct(store)
class(store2)<-("numeric")
Ephemeris$Unix<-store2
Ephemeris<-Ephemeris[,c("Unix", "Elev")]
weather<-merge(weather, Ephemeris, all=TRUE)
weather<-weather[order(weather$Unix),]
weather$dswrf<-ifelse(weather$Elev<=0, 0, weather$dswrf)

In [41]:
MODASfolder<-paste(basefolder, "/modas/", sep="")
MODASfiles<-list.files(path=MODASfolder, pattern=NULL, full.names=FALSE, recursive=FALSE)
for(c in 1:length(MODASfiles)){
    sst<-read.table(paste(MODASfolder, MODASfiles[c], sep=""), header=TRUE)
    sst<-round(sst, digits=2)
    assign(MODASfiles[c], sst)
}
sst<-do.call("rbind", lapply(MODASfiles, as.name))
sst<-sst[,c("Unix", "Temp")]
colnames(sst)[2]<-"sst"
weather<-merge(weather, sst, all=TRUE)

In [42]:
weather<-weather[,c("Unix", "air", "dlwrf", "prate", "pres", "rhum", "wnd", "dswrf", "sst")]

x<-weather$Unix
y<-weather$air
a<-approx(x,y,x, rule=2)
weather$air<-a$y

y<-weather$dlwrf
lw<-approx(x,y,x, rule=2)
weather$dlwrf<-lw$y

y<-weather$prate
pt<-approx(x,y,x, rule=2)
weather$prate<-pt$y

y<-weather$pres
pr<-approx(x,y,x, rule=2)
weather$pres<-pr$y

y<-weather$rhum
rh<-approx(x,y,x, rule=2)
weather$rhum<-rh$y

y<-weather$wnd
wd<-approx(x,y,x, rule=2)
weather$wnd<-wd$y

y<-weather$dswrf
sol<-approx(x,y,x, rule=2)
weather$dswrf<-sol$y

y<-weather$sst
wt<-approx(x,y,x, rule=2)
weather$sst<-wt$y
weather$sst<-weather$sst+273

weather[,c("air","dlwrf", "pres", "wnd", "dswrf", "sst")]<-round(weather[,c("air","dlwrf", "pres", "wnd", "dswrf", "sst")], digits=2)
weather$prate<-round(weather$prate, digits=5)
weather$rhum<-round(weather$rhum, digits=0)

In [43]:
wavesfolder<-paste(basefolder, "/waves/", sep="")
wavesfiles<-list.files(path=wavesfolder, pattern=NULL, full.names=FALSE, recursive=FALSE)

    for(d in 1:length(wavesfiles)){
        w<-read.table(paste(wavesfolder, wavesfiles[d], sep=""), head=TRUE, stringsAsFactors=FALSE)
        w$dp<-ifelse(w$dp>360, NA, w$dp)
        assign(wavesfiles[d], w)
    }

Wave<-do.call("rbind", lapply(wavesfiles, as.name))
x<-Wave$Unix
y<-Wave$dp
dp<-approx(x,y,x, rule=2)
Wave$dp<-dp$y
Wave$dp<-round(Wave$dp, digits=2)
Wave$hs<-as.numeric(Wave$hs)

slope<-incline
slope<-tan(slope*(pi/180))
Wave$L<-(9.8*(Wave$tp^2))/(2*pi)
Wave$I<-slope/sqrt(Wave$hs/Wave$L)
Wave$I<-ifelse(Wave$hs==0, 0, Wave$I)
Wave$Runup<-Wave$hs*0.60*(Wave$I^0.34)
Wave<-Wave[,c("Unix", "Runup", "dp")]

In [44]:
tides<-read.table(paste(basefolder, "/tides/tides_MLLW_", site, ".out", sep=""), header=TRUE)
tides$DateTime<-paste(tides$Date, tides$Time)
store<-strptime(tides$DateTime, "%m.%d.%Y %H:%M")
store2<-as.POSIXct(store)
class(store2)<-("numeric")
tides$Unix<-store2
tides<-tides[,c("Unix", "Tide")]

In [45]:
WaveTide<-merge(Wave, tides, all=TRUE)
x<-WaveTide$Unix

y<-WaveTide$Runup
ru<-approx(x,y,x, rule=2)
WaveTide$Runup<-ru$y

dir<-WaveTide$dp
d<-approx(x,dir,x, rule=2)
WaveTide$dp<-d$y

site_dir<-read.table(paste("../data/WaveDirection.txt", sep=""), header=TRUE)
a<-site_dir$Direction_a[which(site==site_dir$Site)]
b<-site_dir$Direction_b[which(site==site_dir$Site)]

if(b>a){WaveTide$Runup<-ifelse(WaveTide$dp>a & WaveTide$dp<b, WaveTide$Runup, 0)}
if(b<a){WaveTide$Runup<-ifelse(WaveTide$dp>a | WaveTide$dp<b, WaveTide$Runup, 0)}

WaveTide$Runup<-round(WaveTide$Runup, digits=3)

WaveTide$Tide<-WaveTide$Tide+WaveTide$Runup

In [46]:
heights <- seq(0.5, 3.5, 0.1)

In [53]:
heights <-0.5

In [54]:
for(h in 1:length(heights)){
    height <- heights[h]
    height2<-height*100
    newdir <- paste("../modelinput/", site, "/height", sprintf("%03.0f", height2), "cm", sep="")
    dir.create(newdir, showWarnings = FALSE)
    print(site)
    print(height)
    WaveTide$tideflag<-as.numeric(WaveTide$Tide<height)
    TideFlag<-WaveTide[,c("Unix","tideflag")]    
    NOAH<-merge(weather, TideFlag)
    NOAH<-NOAH[order(NOAH$Unix),]
    
    DateConvert<-NOAH$Unix
    class(DateConvert)<-"POSIXct"
    DateExtract<-as.POSIXlt(DateConvert)
    NOAH$hhmm<-as.character(DateExtract, format="%H%M")
    class(NOAH$hhmm)<-"numeric"
    NOAH$year<-as.character(DateExtract, format="%Y")
    class(NOAH$year)<-"numeric"
    allyears<-unique(NOAH$year)
    
    for(y in 1:length(allyears)){
        final<-subset(NOAH, NOAH$year==allyears[y])
        DateConvert<-final$Unix
        class(DateConvert)<-"POSIXct"
        DateExtract<-as.POSIXlt(DateConvert)
        final$jday<-as.character(DateExtract, format="%j")
        class(final$jday)<-"numeric"
        final<-final[order(final$Unix),]
        final<-unique(final)
        final<-final[,c("jday", "hhmm", "wnd", "air", "rhum", "pres", "dswrf", "dlwrf", "prate", "sst", "tideflag")]
        print(site) 
        print(allyears[y])
        print(nrow(final))       
        #write.table(final, paste("../modelinput/", site, "/height", sprintf("%03.0f", height2), "cm/", site, "_", sprintf("%03.0f", height2), "cm_", allyears[y], ".in", sep=""), quote=FALSE, row.names=FALSE, col.names=FALSE)
    }
}

[1] "NZSIBT"
[1] 0.5
[1] "NZSIBT"
[1] 1990
[1] 17520
[1] "NZSIBT"
[1] 1991
[1] 17520
[1] "NZSIBT"
[1] 1992
[1] 17568
[1] "NZSIBT"
[1] 1993
[1] 17520
[1] "NZSIBT"
[1] 1994
[1] 17520
[1] "NZSIBT"
[1] 1995
[1] 17520
[1] "NZSIBT"
[1] 1996
[1] 17568
[1] "NZSIBT"
[1] 1997
[1] 17520
[1] "NZSIBT"
[1] 1998
[1] 17520
[1] "NZSIBT"
[1] 1999
[1] 17520
[1] "NZSIBT"
[1] 2000
[1] 17568
[1] "NZSIBT"
[1] 2001
[1] 17520
[1] "NZSIBT"
[1] 2002
[1] 17520
[1] "NZSIBT"
[1] 2003
[1] 17520
[1] "NZSIBT"
[1] 2004
[1] 17568
[1] "NZSIBT"
[1] 2005
[1] 17520
[1] "NZSIBT"
[1] 2006
[1] 17520
[1] "NZSIBT"
[1] 2007
[1] 17520
[1] "NZSIBT"
[1] 2008
[1] 17568
[1] "NZSIBT"
[1] 2009
[1] 17520
